In [5]:
from pathlib import Path
import pandas as pd
import time
import ast

from model_client import ModelClient, get_prompt
from pdf_extraction import extract_text_from_pdf

In [6]:
pdf_path = Path("./data/pdfs")
client = ModelClient()

responses = []
times = []
for pdf in pdf_path.glob("*.pdf"):
    start_time = time.time()

    paper_text = extract_text_from_pdf(pdf)
    prompt = get_prompt(paper_text)
    response = client.ask_claude(prompt)
    responses.append(response)

    end_time = time.time()
    times.append(end_time - start_time)

In [49]:
import re

cleaned_responses = []
pattern = r'(.*\})\s*(.*)'

for r in responses:
    match = re.match(pattern, r, re.DOTALL)
    if match:
        dict_part = match.group(1)
        notes_part = match.group(2)
        cleaned_r = ast.literal_eval(dict_part)
        cleaned_r["Note"] = notes_part.strip()
        cleaned_responses.append(cleaned_r)

In [ ]:
# results = pd.DataFrame(cleaned_responses)
# save_path = Path("./data/results/claude_results.csv")
# results.to_csv(save_path, index=False)

In [55]:
sum(times) / len(times)

18.00835068821907